In [1]:
import warnings
warnings.filterwarnings("ignore")

import mbuild as mb
from mbuild import Compound
from mbuild.lib.recipes import Polymer

import unyt as u

/home/chris/miniconda3/envs/mosdef/lib/python3.11/site-packages/mdtraj/formats/__init__.py:6: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from .xtc import XTCTrajectoryFile
/home/chris/miniconda3/envs/mosdef/lib/python3.11/site-packages/mdtraj/geometry/order.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import parse_version
/home/chris/miniconda3/envs/mosdef/lib/python3.11/site-packages/pkg_resources/__init__.py:2832: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/chris/miniconda3/envs/mosdef/lib/python3.11/site-packages/lark/utils.py:163: DeprecationWarning: module 'sre_par

# mBuild:

### Goal: Create an initial system of a bead-spring polymer model.
- This is a coarse-grained representation of polymers where we are using abstract beads instead of real atoms or molecules.

### Process:
- Create a custom class that builds up a single bead-spring chain each time it is instantiated.
    - This class should have the flexibility to build chains of different lengths and to set parameters such as the bond length and bead name.
- Find the volume that corresponds to a target number density
- Using `mb.packing.fill_box` to create an initial system of a set of bead-spring chains

In [2]:
class BeadSpringChain(Compound):
    """"""
    def __init__(self, n_beads, bond_length, bead_name, bead_mass):
        self.n_beads = n_beads
        self.bond_length = bond_length
        self.bead_name = bead_name
        self.bead_mass = bead_mass
        super(BeadSpringChain, self).__init__()
        last_bead = None
        for i in range(n_beads):
            bead = mb.Compound(name=bead_name, mass=bead_mass)
            bead.translate([(i*bond_length), 0, 0])
            self.add(bead)
            if last_bead:
                self.add_bond([last_bead, bead])
            last_bead = bead

In [3]:
chain = BeadSpringChain(
    n_beads=20,
    bond_length=1.1,
    bead_mass=1.0,
    bead_name="A"
)

chain.visualize(bead_size=3.5)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
beads_per_chain = 20
num_chains = 30
n_beads = beads_per_chain * num_chains
num_density = 0.70

volume = n_beads / num_density
box_L = volume**(1/3)

box = mb.Box([box_L * 5, box_L * 5, box_L * 5])

In [5]:
chain = BeadSpringChain(
    n_beads=beads_per_chain,
    bond_length=1.1,
    bead_mass=1.0,
    bead_name="A"
)

system = mb.fill_box(
    compound=chain,
    n_compounds=num_chains,
    box=box,
    edge=1,
    overlap=3
)

system.visualize(bead_size=3)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# GMSO:



In [22]:
import gmso
from gmso.external import from_mbuild
from gmso.lib.potential_templates import PotentialTemplateLibrary
from gmso.core.forcefield import ForceField

In [23]:
gaff_path = "/home/chris/cme/forks/gmso/gmso/utils/files/gmso_xmls/test_ffstyles/gaff.xml"
gaff = ForceField(gaff_path)

/home/chris/miniconda3/envs/mosdef/lib/python3.11/site-packages/pydantic/main.py:171: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.__pydantic_validator__.validate_python(data, self_instance=self)
/home/chris/miniconda3/envs/mosdef/lib/python3.11/site-packages/pydantic/main.py:171: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.__pydantic_validator__.validate_python(data, self_instance=self)
/home/chris/miniconda3/envs/mosdef/lib/python3.11/site-packages/pydantic/main.py:171: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your 

In [27]:
help(ForceField)

Help on class ForceField in module gmso.core.forcefield:

class ForceField(builtins.object)
 |  ForceField(xml_loc=None, strict=True, greedy=True, backend='forcefield-utilities')
 |  
 |  A generic implementation of the forcefield class.
 |  
 |  The ForceField class is one of the core data structures in gmso, which is
 |  used to hold a collection of gmso.core.Potential subclass objects along with some
 |  metadata to represent a forcefield. The forcefield object can be applied
 |  to any gmso.Topology which has effects on its Sites, Bonds, Angles and Dihedrals.
 |  
 |  Parameters
 |  ----------
 |  xml_loc : str
 |      Path to the forcefield xml. The forcefield xml can be either in Foyer or GMSO style.
 |  strict: bool, default=True
 |      If true, perform a strict validation of the forcefield XML file
 |  greedy: bool, default=True
 |      If True, when using strict mode, fail on the first error/mismatch
 |  backend: str, default="forcefield-utilities"
 |      Can be "gmso" or "f

In [25]:
type(gaff)

gmso.core.forcefield.ForceField

In [14]:
lib = PotentialTemplateLibrary()

In [20]:
lib["LennardJonesPotential"]

<PotentialTemplate LennardJonesPotential,
 expression: 4*epsilon*(-sigma**6/r**6 + sigma**12/r**12),
 id: 135800595593920>

In [17]:
lib.get_available_template_names()

('PeriodicTorsionPotential',
 'HarmonicAnglePotential',
 'BuckinghamPotential',
 'OPLSTorsionPotential',
 'FixedBondPotential',
 'RyckaertBellemansTorsionPotential',
 'LAMMPSHarmonicAnglePotential',
 'FixedAnglePotential',
 'HarmonicBondPotential',
 'HarmonicImproperPotential',
 'LAMMPSHarmonicBondPotential',
 'FourierTorsionPotential',
 'MiePotential',
 'HarmonicTorsionPotential',
 'PeriodicImproperPotential',
 'LennardJonesPotential')

In [127]:
class LJBeadType(gmso.core.atom_type.AtomType):
    def __init__(self, sigma=1, epsilon=1, name="A"):
        parameters=dict(
            epsilon=epsilon*u.dimensionless,
            sigma=sigma*u.dimensionless
        )
        super(LJBeadType, self).__init__(
            expression="4*epsilon*(-sigma**6/r**6 + sigma**12/r**12)",
            atomclass=name,
            parameters=parameters,
            independent_variables=set("r"),
            name="LennardJonesPotential",
        )


class HarmonicBondType(gmso.core.bond_type.BondType):
    def __init__(self, k=100, r_eq=1, bond_name="A-A"):
        parameters=dict(
            k=k*u.dimensionless,
            r_eq=r_eq*u.dimensionless
        )
        members = tuple(bond_name.split("-"))
        super(HarmonicBondType, self).__init__(
            expression="0.5*k*(r - r_eq)**2",
            name="HarmonicBondPotential",
            parameters=parameters,
            independent_variables=set("r"),
            member_types=members,
            member_classes=members            
        )


class HarmonicAngleType(gmso.core.angle_type.AngleType):
    def __init__(self, k=100, t_eq=2.5*u.radian, angle_name="A-A-A"):
        parameters=dict(
            k=k*u.dimensionless,
            t_eq=t_eq
        )
        members = tuple(angle_name.split("-"))
        super(HarmonicAngleType, self).__init__(
            expression="0.5*k*(t - t_eq)**2",
            name="HarmonicAnglePotential",
            parameters=parameters,
            independent_variables=set("t"),
            member_types=members,
            member_classes=members
        )

In [128]:
gmso_system = from_mbuild(system)
gmso_system.identify_connections()

In [129]:
for site in gmso_system.sites:
    site.atom_type = LJBeadType()

for bond in gmso_system.bonds:
    bond.bond_type = HarmonicBondType(r_eq=1.1)

for angle in gmso_system.angles:
    angle.angle_type = HarmonicAngleType(t_eq=2.2*u.radian)

In [130]:
from gmso.external.convert_hoomd import to_hoomd_snapshot, to_hoomd_forcefield

In [131]:
ff, refs = to_hoomd_forcefield(top=gmso_system, base_units=None, r_cut=2.5)

EngineIncompatibilityError: Potential <LJBeadType LennardJonesPotential, expression: 4*epsilon*(-sigma**6/r**6 + sigma**12/r**12), id: 129582952490048> is not in the list of accepted_potentials (<PotentialTemplate LennardJonesPotential,
 expression: 4*epsilon*(-sigma**6/r**6 + sigma**12/r**12),
 id: 129583076062096>, <PotentialTemplate HarmonicBondPotential,
 expression: 0.5*k*(r - r_eq)**2,
 id: 129583089141232>, <PotentialTemplate HarmonicAnglePotential,
 expression: 0.5*k*(theta - theta_eq)**2,
 id: 129583086160944>, <PotentialTemplate PeriodicTorsionPotential,
 expression: k*(cos(n*phi - phi_eq) + 1),
 id: 129582963350800>, <PotentialTemplate OPLSTorsionPotential,
 expression: 0.5*k1*(cos(phi) + 1) + 0.5*k2*(1 - cos(2*phi)) + 0.5*k3*(cos(3*phi) + 1) + 0.5*k4*(1 - cos(4*phi)),
 id: 129582978868208>, <PotentialTemplate RyckaertBellemansTorsionPotential,
 expression: c0 + c1*cos(phi) + c2*cos(phi)**2 + c3*cos(phi)**3 + c4*cos(phi)**4 + c5*cos(phi)**5,
 id: 129583083313168>)

In [132]:
box = mb.fill_box(
    compound=mb.load("C", smiles=True),
    n_compounds=1,
    box=mb.Box([1, 1, 1])
)
top = from_mbuild(box)
opls = gmso.core.forcefield.ForceField("oplsaa")
apply(top=top, identify_connections=True, forcefields=opls)

<Topology Topology, 5 sites,
 10 connections,
 15 potentials,
 id: 129582983431120>

In [133]:
atom_types = [site.atom_type for site in top.sites]

In [142]:
atom_types[0]

<AtomType opls_138,
 expression: 4*epsilon*(-sigma**6/r**6 + sigma**12/r**12),
 id: 129583124184784,
 atomclass: CT>